In [12]:
import pandas as pd

df = pd.read_csv("steam_games_dataset_kaggle.csv")
df = df.dropna()  # This will drop rows that contain any NaN values

#variables we will use
df1= df[["name", "desc_snippet","genre"]]

#delete missing values
df2 = pd.DataFrame(df1.dropna())

df = df2.head(5000)

# Reset the index of the DataFrame
df = df.reset_index(drop=True)

df.to_csv('steam_games_dataset_kaggle_5000_clean.csv', index=False)
df.head()

,name,desc_snippet,genre
0,Devil May Cry 5,"The ultimate Devil Hunter is back in style, in...",Action
1,Life is Strange 2,"After a tragic incident, brothers Sean and Dan...",Adventure
2,Call of Duty®: WWII,Call of Duty® returns to its roots with Call o...,Action
3,Shadow Tactics: Blades of the Shogun,Shadow Tactics is a hardcore tactical stealth ...,"Indie,Strategy"
4,SCUM,SCUM aims to evolve the multiplayer open world...,"Action,Adventure,Indie,Massively Multiplayer,E..."


In [13]:
# Extract unique genres from the Kaggle dataset
unique_genres = set([genre for genres in df['genre'] for genre in genres.split(',')])

# Print the unique genres from the Kaggle dataset
print(unique_genres)

{'Casual', 'Strategy', 'Simulation', 'Early Access', 'Indie', 'RPG', 'Action', 'Massively Multiplayer', 'Adventure', 'Free to Play'}


In [14]:
# Create binary columns for each unique genre
for genre in unique_genres:
    df[genre] = df['genre'].apply(lambda x: int(genre in x))

# Print the updated dataset
df.head()

,name,desc_snippet,genre,Casual,Strategy,Simulation,Early Access,Indie,RPG,Action,Massively Multiplayer,Adventure,Free to Play
0,Devil May Cry 5,"The ultimate Devil Hunter is back in style, in...",Action,0,0,0,0,0,0,1,0,0,0
1,Life is Strange 2,"After a tragic incident, brothers Sean and Dan...",Adventure,0,0,0,0,0,0,0,0,1,0
2,Call of Duty®: WWII,Call of Duty® returns to its roots with Call o...,Action,0,0,0,0,0,0,1,0,0,0
3,Shadow Tactics: Blades of the Shogun,Shadow Tactics is a hardcore tactical stealth ...,"Indie,Strategy",0,1,0,0,1,0,0,0,0,0
4,SCUM,SCUM aims to evolve the multiplayer open world...,"Action,Adventure,Indie,Massively Multiplayer,E...",0,0,0,1,1,0,1,1,1,0


In [15]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string

# Download NLTK resources (required for tokenization and stopwords)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Preprocessing function
def preprocess_text(text):
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Stopword removal and lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    # Remove special characters and punctuation
    tokens = [token for token in tokens if token not in string.punctuation]
    
    # Join tokens back into text
    processed_text = ' '.join(tokens)
    
    return processed_text

# Apply preprocessing to the 'description' column
df['processed_desc_snippet'] = df['desc_snippet'].apply(preprocess_text)

# Save the updated DataFrame to a new CSV file
df.to_csv('steam_games_dataset_kaggle_5000_clean_binary_genre.csv.csv', index=False)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [52]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

df = pd.read_csv("preprocessed_kaggle_multi_labels2.csv")

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['processed_desc_snippet'])
y = df.iloc[:, 3:-1]  # Output labels (binary genre columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [53]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Convert X_train and y_train to NumPy arrays
X_train = X_train.toarray()
y_train = y_train.values

# Create a sequential model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test.toarray(), y_test.values))

# Evaluate the model
loss, accuracy = model.evaluate(X_test.toarray(), y_test.values)
print("Accuracy:", accuracy)


Epoch 1/100
3/3 [==============================] - 1s 56ms/step - loss: 0.7005 - accuracy: 0.1692 - val_loss: 0.6817 - val_accuracy: 0.1176
Epoch 2/100
3/3 [==============================] - 0s 14ms/step - loss: 0.6663 - accuracy: 0.2000 - val_loss: 0.6662 - val_accuracy: 0.1176
Epoch 3/100
3/3 [==============================] - 0s 14ms/step - loss: 0.6350 - accuracy: 0.2615 - val_loss: 0.6516 - val_accuracy: 0.1176
Epoch 4/100
3/3 [==============================] - 0s 16ms/step - loss: 0.6092 - accuracy: 0.2923 - val_loss: 0.6372 - val_accuracy: 0.1176
Epoch 5/100
3/3 [==============================] - 0s 14ms/step - loss: 0.5825 - accuracy: 0.2615 - val_loss: 0.6221 - val_accuracy: 0.1176
Epoch 6/100
3/3 [==============================] - 0s 14ms/step - loss: 0.5528 - accuracy: 0.3385 - val_loss: 0.6066 - val_accuracy: 0.1176
Epoch 7/100
3/3 [==============================] - 0s 14ms/step - loss: 0.5150 - accuracy: 0.3846 - val_loss: 0.5912 - val_accuracy: 0.1176
Epoch 8/100
3/3 [===